
## Zadania do wykonania

### Zadanie 1

Napisz program typu `CRUD` zbierający sygnały (zdarzenia) z różnych źródeł. Tabela powinna posiadać:

- dokładną datę zdarzenia,
- źródło zdarzenia,
- typ zdarzenia (informacja, ostrzeżenie, błąd, błąd krytyczny),
- dodatkowe dane,
- identyfikator (adres może być np. IP).

Na kolumnę typ zdarzenia powinien zostać założony indeks, a wartości powinny być ograniczone do tych wyliczonych w nawiasie.


In [ ]:
#r "nuget: FluentNHibernate, 3.1.0"
#r "nuget: System.Data.SQLite, 1.0.113.7"

using System;
using NHibernate;
using FluentNHibernate;

// Create BuildSchema
private static void BuildSchema(Configuration config)
{
    new SchemaExport(config)
            .Create(false, true);
}

// Session factory
private static ISessionFactory CreateSessionFactory(params Type[] mappingTypes)
{
    return Fluently.Configure()
                        .Database(SQLiteConfiguration.Standard
                            .UsingFile("database.sqlite"))
                        .Mappings(m => {
                            foreach(var mappingType in mappingTypes) 
                                m.FluentMappings.Add(mappingType);
                        })
                        .ExposeConfiguration(BuildSchema)
                        .BuildSessionFactory();
}

public enum EventType
{
    Information,
    Warning,
    Error,
    CriticalError
}

// Model
public class Record
{
    public virtual int Id { get; set; }
    public virtual EventType EventType { get; set; }
    public virtual DateTime EventTime { get; set; } 
    public virtual string Source { get; set; }
    public virtual string AdditionalData { get; set; }

    public override string ToString()
    {
        return string.Format("Id: {0}, EventType: {1}, EventTime: {2}, Source: {3}, AdditionalData: {4}", Id, EventType, EventTime, Source, AdditionalData);
    }
}

// Refenced from here: https://github.com/nhibernate/nhibernate-core/blob/master/src/NHibernate/UserTypes/IUserType.cs
public class EventTypeUserType : IUserType
{
    public SqlType[] SqlTypes => new[] { SqlTypeFactory.GetString(255) };

    public Type ReturnedType => typeof(EventType);

    public new bool Equals(object x, object y) => object.Equals(x, y);

    public int GetHashCode(object x) => x.GetHashCode();

    public object NullSafeGet(DbDataReader rs, string[] names, ISessionImplementor session, object owner)
    {
        var value = NHibernateUtil.String.NullSafeGet(rs, names[0], session);
        return Enum.Parse(typeof(EventType), value.ToString());
    }

    public void NullSafeSet(DbCommand cmd, object value, int index, ISessionImplementor session)
    {
        var parameter = (IDataParameter)cmd.Parameters[index];
        parameter.Value = value.ToString();
    }

    public object DeepCopy(object value) => value;

    public object Replace(object original, object target, object owner) => original;

    public object Assemble(object cached, object owner) => cached;

    public object Disassemble(object value) => value;

    public bool IsMutable => false;
}

// Mapping class
public class RecordMap : ClassMap<Record>
{
    public RecordMap()
    {
        Table("Record");

        Id(x => x.Id)
            .GeneratedBy.Identity();
        Map(x => x.EventType)
            .CustomType<EventTypeUserType>()
            .Column("EventType")
            .Not.Nullable();
        Map(x => x.EventTime).Not.Nullable();
        Map(x => x.Source).Not.Nullable();
        Map(x => x.AdditionalData).Not.Nullable();
    }
}

// Following the structure similar to Spring applications
public static class SessionManager
{

    public static void BuildSchema(Configuration config)
    {
        new SchemaExport(config)
                .Create(false, true);
    }


    public static ISessionFactory CreateSessionFactory(params Type[] mappingTypes)
    {
        return Fluently.Configure()
                            .Database(SQLiteConfiguration.Standard
                                .UsingFile("database.sqlite"))
                            .Mappings(m => {
                                foreach(var mappingType in mappingTypes) 
                                    m.FluentMappings.Add(mappingType);
                            })
                            .ExposeConfiguration(BuildSchema)
                            .BuildSessionFactory();
    }
}

public class RecordRepository
{
    private readonly ISessionFactory sessionFactory;
    private readonly ISession session;

    public RecordRepository(ISessionFactory sessionFactory)
    {
        this.sessionFactory = sessionFactory;
        this.session = sessionFactory.OpenSession();
    }

    public void Create(Record record)
    {
        using (var transaction = session.BeginTransaction())
        {
            try
            {
                session.Save(record);
                transaction.Commit();
            }
            catch (Exception ex)
            {
                transaction.Rollback();
                throw ex;
            }
        }
    }

    public void Update(Record record)
    {
        using (var transaction = session.BeginTransaction())
        {
            try
            {
                session.Update(record);
                transaction.Commit();
            }
            catch (Exception ex)
            {
                transaction.Rollback();
                throw ex;
            }
        }
    }

    public void FindAll()
    {
        using (var transaction = session.BeginTransaction())
        {
            try
            {
                var records = session.Query<Record>().ToList();
                foreach (var record in records)
                {
                    Console.WriteLine(record);
                }
                transaction.Commit();
            }
            catch (Exception ex)
            {
                transaction.Rollback();
                throw ex;
            }
        }
    }

    public void Delete(Record record)
    {
        using (var transaction = session.BeginTransaction())
        {
            try
            {
                // find if record exists in fatabase
                if (session.Get<Record>(record.Id) == null)
                {
                    throw new Exception("Record does not exist in database");
                }
                else 
                {
                    session.Delete(record);
                    transaction.Commit();
                }
            }
            catch (Exception ex)
            {
                transaction.Rollback();
                throw ex;
            }
        }
    }


}

class RecordApi
{

    RecordRepository recordRepository = new RecordRepository(CreateSessionFactory(typeof(RecordMap)));

    public void AddRecord(Record record)
    {
        recordRepository.Create(record);
    }

    public void updateRecord(Record record)
    {
        recordRepository.Update(record);
    }

    public void findAllRecords()
    {
        recordRepository.FindAll();
    }

    public void deleteRecord(Record record)
    {
        recordRepository.Delete(record);
    }

}

RecordApi recordApi = new RecordApi();

Record record = new Record() { EventType = EventType.Information, EventTime = DateTime.Now, Source = "Source", AdditionalData = "AdditionalData" };

recordApi.AddRecord(record);
recordApi.findAllRecords();
recordApi.deleteRecord(record);
recordApi.findAllRecords();


### Zadanie 2

Do zadania 2 z laboratorium 1 dodaj obsługę bazy danych.


In [ ]:
#r "nuget: FluentNHibernate, 3.1.0"
#r "nuget: System.Data.SQLite, 1.0.113.7"

using System;
using NHibernate;
using FluentNHibernate;


class CatalogGeneric<T> where T : Employee
{
    private List<T> catalog = [];

    private EmployeeDTOApi employeeDTOApi = new EmployeeDTOApi();

    public void Add(T item) { catalog.Add(item);}
    public void AddMany(List<T> items) { catalog.AddRange(items); }
    public void Remove(T item) { catalog.Remove(item); }
    public void Update(T item, Employee newItem) { catalog[catalog.IndexOf(item)] = (T)newItem; }

    public void Print()
    {
        if (catalog.Count == 0)
        {
            Console.WriteLine("Catalog is empty");
        }
        else
        {
            foreach (T item in catalog)
            {
                Console.WriteLine(item?.ToString());
            }
        }
    }

    public T? Search(T catalogItem) { return (catalog.Count != 0 && catalog.Contains(catalogItem)) ? catalogItem : default(T); }
    public void Clear() { catalog.Clear(); }
    public bool Validate(T item) { return catalog.Contains(item); }

    public EmployeeDTO mapEmployeeToEmployeeDTO(Employee employee)
    {
        return new EmployeeDTO() { Name = employee.Name, Surname = employee.Surname, Position = employee.Position, Email = employee.Email, Id = employee.Id };
    }

    public Employee mapEmployeeDTOToEmployee(EmployeeDTO employeeDTO)
    {
        return new Employee() { Name = employeeDTO.Name, Surname = employeeDTO.Surname, Position = employeeDTO.Position, Email = employeeDTO.Email, Id = employeeDTO.Id };
    }

    public void AddToDatabase(T item)
    {
        if (item != null)
        {
            if (item is Employee)
            {
                Employee employee = item;
                if (employee.ValidateData())
                {
                    employee = mapEmployeeDTOToEmployee(employeeDTOApi.AddEmployeeDTO(mapEmployeeToEmployeeDTO(employee)));
                }
                else
                {
                    Console.WriteLine("Employee did not validate");
                }

                Update(item, employee);
            }
        }
        else
        {
            Console.WriteLine("Item is null");
        }
    }

    // read all from database
    public List<EmployeeDTO> ReadFromDatabase()
    {
        return employeeDTOApi.findAllEmployeeDTOs();

    }

    public void UpdateInDatabase(T item)
    {
        if (item != null)
        {
            if (item is Employee)
            {
                Employee employee = item;
                if (employee.ValidateData())
                {
                    employeeDTOApi.updateEmployeeDTO(mapEmployeeToEmployeeDTO(employee));
                }
                else
                {
                    Console.WriteLine("Employee did not validate");
                }
            }
        }
        else
        {
            Console.WriteLine("Item is null");
        }
    }

    public void RemoveFromDatabase(T item)
    {
        if (item != null)
        {
            if (item is Employee)
            {
                Employee employee = item;
                if (employee.ValidateData())
                {
                    employeeDTOApi.deleteEmployeeDTO(mapEmployeeToEmployeeDTO(employee));
                }
                else
                {
                    Console.WriteLine("Employee did not validate");
                }
            }
        }
        else
        {
            Console.WriteLine("Item is null");
        }
    }

}

public class Employee
{

    public string Name { get; set; }
    public string Surname { get; set; }
    public string Position { get; set; }
    public string Email { get; set; }
    public int Id { get; set; }

    public Employee()
    {
    }

    public Employee(string name, string surname, string position, string email, int id)
    {
        Name = name;
        Surname = surname;
        Position = position;
        Email = email;
        Id = id;
    }

    // Overriding Equals(), otherwise Contains() won't work, GetHashCode() is also required
    public override bool Equals(object obj)
    {
        if (obj == null || GetType() != obj.GetType())
        {
            return false;
        }
        else
        {
            Employee employee = (Employee)obj;
            return (
                this.Name == employee.Name &&
                this.Surname == employee.Surname &&
                this.Position == employee.Position &&
                this.Email == employee.Email &&
                this.Id == employee.Id
            );
        }
    }

    public override int GetHashCode() { return Id.GetHashCode(); }

    public bool ValidateData() { return EmployeeValidator.ValidateAll(this.Name, this.Surname, this.Position, this.Email, this.Id); }

    public String Show() { return string.Format("Name: {0}, Surname: {1}, Position: {2}, Email: {3},  Id: {4}", this.Name, this.Surname, this.Position, this.Email, this.Id); }

    public bool IsMatch(Employee employee) { return employee.Equals(this); }

    public override string ToString() { return string.Format("Name: {0}, Surname: {1}, Position: {2}, Email: {3}, Id: {4}", this.Name, this.Surname, this.Position, this.Email, this.Id); }

}

public class EmployeeValidator
{

    const int MIN_NAME_LENGTH = 2;
    const int MAX_NAME_LENGTH = 20;
    const int MIN_POSITION_LENGTH = 2;
    const int MAX_POSITION_LENGTH = 20;
    const int MIN_ID = 0;
    const String NAME_REGEX = @"^[a-zA-Z]+$";
    const String EMAIL_REGEX = "^[A-Za-z0-9.]+@(.+)$";  // Regex pattern : any number of letters, numbers and dots, then @, 
                                                        // then any number of letters, numbers and dots and $ asserting the end of the string

    // validate name and surname
    private static bool ValidateName(String name)
    {
        return !String.IsNullOrEmpty(name) && name.Length > MIN_NAME_LENGTH && name.Length < MAX_NAME_LENGTH && Regex.IsMatch(name, NAME_REGEX);
    }

    // validate email address against regex
    private static bool ValidateEmail(String email)
    {
        return Regex.IsMatch(email, EMAIL_REGEX);
    }

    private static bool ValidatePosition(String position)
    {
        return !String.IsNullOrEmpty(position) && position.Length > MIN_POSITION_LENGTH && position.Length < MAX_POSITION_LENGTH && Regex.IsMatch(position, NAME_REGEX);
    }

    private static bool ValidateId(int id)
    {
        return id > MIN_ID;
    }

    public static bool ValidateAll(String name, String surname, String position, String email, int id)
    {
        return (
            ValidateName(name) &&
            ValidateName(surname) &&
            ValidatePosition(position) &&
            ValidateEmail(email) &&
            ValidateId(id)
            );
    }
}

public class EmployeeDTO 
{
    public virtual int Id { get; set; }
    public virtual string Name { get; set; }
    public virtual string Surname { get; set; }
    public virtual string Position { get; set; }
    public virtual string Email { get; set; }

    public override string ToString() => $"Id: {Id}, {Name}, {Surname}, {Position}, {Email}";
}

// Mapping class
public class EmployeeDTOMap : ClassMap<EmployeeDTO>
{
    public EmployeeDTOMap()
    {
        Table("Employee");

        Id(x => x.Id);
        Map(x => x.Name).Not.Nullable();
        Map(x => x.Surname).Not.Nullable();
        Map(x => x.Position).Not.Nullable();
        Map(x => x.Email).Not.Nullable();
    }
}

public static class SessionManager
{

    public static void BuildSchema(Configuration config)
    {
        new SchemaExport(config)
                .Create(false, true);
    }


    public static ISessionFactory CreateSessionFactory(params Type[] mappingTypes)
    {
        return Fluently.Configure()
                            .Database(SQLiteConfiguration.Standard
                                .UsingFile("employee_db.sqlite"))
                            .Mappings(m => {
                                foreach(var mappingType in mappingTypes) 
                                    m.FluentMappings.Add(mappingType);
                            })
                            .ExposeConfiguration(BuildSchema)
                            .BuildSessionFactory();
    }
}

public class EmployeeDTORepository
{
    // To have only one session factory and session
    private readonly ISessionFactory sessionFactory;
    private readonly ISession session;

    public EmployeeDTORepository(ISessionFactory sessionFactory)
    {
        this.sessionFactory = sessionFactory;
        this.session = sessionFactory.OpenSession();
    }

    public EmployeeDTO Create(EmployeeDTO employeeDTO)
    {
        using (var transaction = session.BeginTransaction())
        {
            try
            {
                session.Save(employeeDTO);
                var addedEmployee = session.Query<EmployeeDTO>().Where(e => e.Name == employeeDTO.Name && e.Surname == employeeDTO.Surname && e.Position == employeeDTO.Position && e.Email == employeeDTO.Email && e.Id == employeeDTO.Id).SingleOrDefault();
                transaction.Commit();

                return addedEmployee;
            }
            catch (Exception ex)
            {
                transaction.Rollback();
                throw ex;
            }
        }
    }

    public void Update(EmployeeDTO employeeDTO)
    {
        using (var transaction = session.BeginTransaction())
        {
            try
            {
                session.Update(employeeDTO);
                transaction.Commit();
            }
            catch (Exception ex)
            {
                transaction.Rollback();
                throw ex;
            }
        }
    }

    public List<EmployeeDTO> FindAll()
    {
        using (var transaction = session.BeginTransaction())
        {
            try
            {
                var employeeDTOs = session.Query<EmployeeDTO>().ToList();
                transaction.Commit();

                return employeeDTOs;
            }
            catch (Exception ex)
            {
                transaction.Rollback();
                throw ex;
            }
        }
    }

    public void Delete(EmployeeDTO employeeDTO)
    {
        using (var transaction = session.BeginTransaction())
        {
            try
            {
                // find if employeeDTO exists in database
                if (session.Get<EmployeeDTO>(employeeDTO.Id) == null)
                {
                    Console.WriteLine("Employee does not exist in database");
                }
                else 
                {
                    session.Delete(session.Get<EmployeeDTO>(employeeDTO.Id));
                    transaction.Commit();
                }
            }
            catch (Exception ex)
            {
                transaction.Rollback();
                throw ex;
            }
        }
    }

    public EmployeeDTO findById(int id)
    {
        using (var transaction = session.BeginTransaction())
        {
            try
            {
                var employeeDTO = session.Query<EmployeeDTO>().Where(e => e.Id == id).SingleOrDefault();
                transaction.Commit();

                return employeeDTO;
            }
            catch (Exception ex)
            {
                transaction.Rollback();
                throw ex;
            }
        }
    }

}

public class EmployeeDTOApi
{
    EmployeeDTORepository employeeDTORepository = new EmployeeDTORepository(SessionManager.CreateSessionFactory(typeof(EmployeeDTOMap)));

    public EmployeeDTO AddEmployeeDTO(EmployeeDTO employeeDTO)
    {
        return employeeDTORepository.Create(employeeDTO);
    }

    public void updateEmployeeDTO(EmployeeDTO employeeDTO)
    {
        employeeDTORepository.Update(employeeDTO);
    }

    public List<EmployeeDTO> findAllEmployeeDTOs()
    {
        return employeeDTORepository.FindAll();
    }

    public void deleteEmployeeDTO(EmployeeDTO employeeDTO)
    {
        employeeDTORepository.Delete(employeeDTO);
    }

    public EmployeeDTO findById(int id)
    {
        return employeeDTORepository.findById(id);
    }

}

CatalogGeneric<Employee> catalog = new CatalogGeneric<Employee>();

Employee employee = new Employee("Eskel", "Wiedzmin", "Wiedzmin", "eskel@gmail.com", 1);

catalog.Add(employee);
catalog.AddToDatabase(employee);
foreach (EmployeeDTO employeeDTO in catalog.ReadFromDatabase())
{
    Console.WriteLine(employeeDTO.ToString());
}
catalog.RemoveFromDatabase(employee);



### Zadanie 3

Napisz klasę do logowania błędów. Zdarzenia powinny być logowane do bazy danych jako *bulk load*. Klasa powinna obsługiwać dwie bazy danych - główną (dowolna relacyjna baza danych) i lokalna (na dane tymczasowe). Schemat logowania jest następujący:
- jeśli baza danych (operacyjna) jest niedostępna logowanie powinno nastąpić do lokalnej bazy danych,
- jeśli połączenie do głównej bazy głównej zostanie przywrócone, klasa powinna przenieść wszystkie dane z lokalnej bazy danych do głównej.

Należy pamiętać o unikalności klucza głównego.

In [ ]:
#r "nuget: FluentNHibernate, 3.1.0"
#r "nuget: System.Data.SQLite, 1.0.113.7"

using System;
using NHibernate;
using FluentNHibernate;

// Model of the ErrorLogger
public class ErrorLog
{
    public virtual int Id { get; set; }
    public virtual string Message { get; set; }

    public override string ToString() => $"Id: {Id}, Message: {Message}";
}

// Mapper for the ErrorLogger
public class ErrorLogMap : ClassMap<ErrorLog>
{
    public ErrorLogMap()
    {
        Table("error_logs");

        Id(x => x.Id).GeneratedBy.Identity();
        Map(x => x.Message).Not.Nullable();
    }
}

// Session manager for creating session factories
public static class SessionManager
{

    public static void BuildSchema(Configuration config)
    {
        new SchemaExport(config)
                .Create(false, true);
    }

    // The parameter is the connection string and the mapping is hardcoded
    public static ISessionFactory CreateSessionFactoryRemote(string connectionString)
    {
        return Fluently.Configure()
                            .Database(SQLiteConfiguration.Standard.ConnectionString(connectionString).ShowSql())
                            .Mappings(m => m.FluentMappings.AddFromAssemblyOf<ErrorLogMap>())
                            .ExposeConfiguration(BuildSchema)
                            .BuildSessionFactory();
    }

    public static ISessionFactory CreateSessionFactoryLocal()
    {
        return Fluently.Configure()
                    .Database(SQLiteConfiguration.Standard
                        .UsingFile("local_db.sqlite"))
                    .Mappings(m => m.FluentMappings.AddFromAssemblyOf<ErrorLogMap>())
                    .ExposeConfiguration(BuildSchema)
                    .BuildSessionFactory();
    }
} 

// The actual class for logging erros
class ErrorLoggingApp
{

    private bool isRemoteDown = false;
    private ISessionFactory remoteSessionFactory;
    private ISessionFactory localSessionFactory;

    public ErrorLoggingApp(string remoteConnectionString)
    {
        this.remoteSessionFactory = SessionManager.CreateSessionFactoryRemote(remoteConnectionString);
        this.localSessionFactory = SessionManager.CreateSessionFactoryLocal();
    }

    // Sync data method
    public void SynchronizeData()
    {
        // Using the stateless session to batch load data : https://nhibernate.info/doc/nhibernate-reference/batch.html
        using (var remoteSession = remoteSessionFactory.OpenStatelessSession())
        {
            using (var localSession = localSessionFactory.OpenStatelessSession())
            {
                using (var transaction = remoteSession.BeginTransaction())
                {
                    try
                    {
                        var localErrors = localSession.QueryOver<ErrorLog>().List();
                        using (var mainTransaction = remoteSession.BeginTransaction())
                        {
                            foreach (var error in localErrors)
                            {
                                remoteSession.Insert(error);
                            }

                            mainTransaction.Commit();
                        }
                        transaction.Commit();
                    }
                    catch (Exception ex)
                    {
                        transaction.Rollback();
                        throw ex;
                    }
                }
            }
        }
    }

    // Error logging
    public void LogData(List<ErrorLog> errors, bool isRemoteDown)
    {
        if (isRemoteDown)
        {
            using (var localSession = localSessionFactory.OpenSession())
            {
                using (var transaction = localSession.BeginTransaction())
                {
                    try
                    {
                        foreach (var error in errors)
                        {
                            localSession.Save(error);
                        }
                        transaction.Commit();
                    }
                    catch (Exception ex)
                    {
                        transaction.Rollback();
                        throw ex;
                    }
                }
            }
        }
        else
        {
            using (var remoteSession = remoteSessionFactory.OpenSession())
            {
                using (var transaction = remoteSession.BeginTransaction())
                {
                    try
                    {
                        foreach (var error in errors)
                        {
                            remoteSession.Save(error);
                        }
                        transaction.Commit();
                        isRemoteDown = false;
                    }
                    catch (Exception ex)
                    {
                        transaction.Rollback();
                        isRemoteDown = true;
                        throw ex;
                    }
                }
            }
        }
    }

    // Watcher for the remote database connection
    public bool CheckRemoteConnection()
    {
        using (var remoteSession = remoteSessionFactory.OpenSession());
        {
            try
            {
                return false;
            }
            catch (Exception ex)
            {
                return true;
            }
        }
    }

    public void LogErrorsWithCheck(List<ErrorLog> errors)
    {
            // Log data
            LogData(errors, isRemoteDown);
            isRemoteDown = CheckRemoteConnection();
            // If remote database is up, synchronize data
            if (!isRemoteDown)
            {
                SynchronizeData();
            }
    }
    
}